# Configuration

## Dependencies

In [ ]:
!python --version

# In Google Colab, all dependencies are already installed
# If not in Colab, uncomment and run the following lines
!pip install --upgrade pip
!pip install pandas google-auth google-cloud-aiplatform google-cloud-bigquery google-cloud-bigquery-storage db-dtypes

## GCP Project and VertexAI

In [ ]:
# Refer to the README for instructions on how to set up the environment
# and authenticate with Google Cloud.
MODEL_NAME   = 'gemini-2.0-flash-lite-001'
GCP_LOCATION = 'us-central1'
GCP_PROJECT  = 'trkkn-cl-int-fm'
LANGUAGE     = 'Spanish'
DATASET_NAME = 'feedgen'
INPUT_TABLE  = 'input_table'
OUTPUT_TABLE = 'output_table'
API_KEY      = ''
BQ_SERVICE_ACCOUNT = ''
VERTEXAI_SERVICE_ACCOUNT = ''

## LLM - Gemini - Configuration

In [ ]:
from vertexai.generative_models import SafetySetting

GENERATION_CONFIG_PRODUCT_ATTRIBUTES_JSON = {
  "temperature": 1.0,
  "top_p": 0.95,
  #"top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
  "response_schema": {"type":"OBJECT","properties":{"response":{"type":"STRING"}}},
}

GENERATION_CONFIG_TITLE_JSON = {
  "temperature": 1.0,
  "top_p": 0.95,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
  "response_schema": { ## JSON output failed when using API_KEY. Preferred to use VertexAI
      "type": "OBJECT",
      "properties": {
        "product attribute keys in original title": {
          "type": "STRING"
        },
        "product category": {
          "type": "STRING"
        },
        "product attribute keys": {
          "type": "STRING"
        },
        "product attribute values": {
          "type": "STRING"
        },
        "replaced keys": {
          "type": "STRING"
        },
        "generated title": {
          "type": "STRING"
        }
      }
    }
}

GENERATION_CONFIG_DESCRIPTION_JSON = {
  "temperature": 1.0,
  "top_p": 0.95,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
  "response_schema": { ## JSON output failed when using API_KEY. Preferred to use VertexAI
      "type": "OBJECT",
      "properties": {
        "description": {
          "type": "STRING"
        },
        "score": {
          "type": "INTEGER"
        },
        "reasoning": {
          "type": "STRING"
        }
      }
    }
}

GENERATION_CONFIG_PRODUCT_ATTRIBUTES_TEXT = {
  "temperature": 1.0,
  "top_p": 0.95,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain"
}

GENERATION_CONFIG_TITLE_TEXT = {
  "temperature": 1.0,
  "top_p": 0.95,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain"
}

GENERATION_CONFIG_DESCRIPTION_TEXT = {
  "temperature": 1.0,
  "top_p": 0.95,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain"
}

SAFETY_SETTINGS = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]

## Authentication (Colab)

In [ ]:
# If you are in Google Colab, just do:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

## Authentication (ADC: Application Default Credentials)

In [ ]:
## If you are not in Google Colab, execute the following code to authenticate in the terminal:
#gcloud auth login
#gcloud config set project $GCP_PROJECT

## Then, run the following code to check the authentication:
#from google.auth import default
#credentials, project_id = default()
#credentials, project_id

# Gen4AIFeeds Library

## Prompts

In [ ]:
# %%
# Original feedgen prompts from github
title_system_instruction = '''You are a leading digital marketer working for a top retail organisation. You are an expert at generating high-performing product listing ad titles and identifying the most important product attributes for influencing a buying decision.
Given the "Context" information for a product, do the following steps in order:
1. Generate "product attribute keys in original title", "product category", "product attribute keys" and "product attribute values" in Spanish. Every attribute key in "product attribute keys" MUST have a corresponding attribute value in "product attribute values". Use whitespaces to represent empty attribute values.
2. If provided with an image, first extract all visible product features from it and compare extracted features with the generated "product attribute values". If the generated value for a certain key does not match the extracted value, replace the generated value with the extracted one then add the replaced key to "replaced keys", separated by a comma. Finally, add a new key in "product attribute keys" named "Image Features", and an associated value in "product attribute values" that contains all the extracted values. You MUST output "Image Features" if an image is provided.
3. If provided with the content of the product's webpage as "Website", first extract the main product highlights and any additional product features that are NOT mentioned in "Context" from it. Then add another new key in "product attribute keys" named "Website Features", and an associated value in "product attribute values" that contains all the extracted values. You MUST output "Website Features" if "Website" information is provided.
4. Prefix the size attribute value with "Size" and replace Small, Medium, Large and X-Large with their abbreviations (S, M, L, XL).
4.b. **Contextualize Numeric Values:** Before concatenating, review each "product attribute value". If a value is purely numeric (e.g., "24", "330", "5.1") and its meaning is not immediately clear without its corresponding key or a unit, you MUST prepend or append a concise descriptor or unit from its "product attribute key" or the original context to make it understandable. For example, if "product attribute key" is "Cantidad" and value is "24", consider "24 unidades" or "24x". If key is "Volumen" and value is "330", use "330ml" or "330cc". If key is "Graduación Alcohólica" and value is "5.1", use "5.1°" or "5.1% Alc.". The goal is clarity in the final title.
5. Separate attribute values with commas.
6. Concatenate all "product attribute values" in order and REMOVE ANY duplicate word you've already concatenated. The length of the final value MUST NOT exceed 150 characters, and it MUST NOT contain any duplicate words - THIS IS VERY IMPORTANT. Output it on a new line, prefixed with "generated title: ".
7. The title should not contain attributes that are not useful. For example, if the product is not available in different colors, the title should not include "Color: Red, Blue".
8. The title should not contain attributes that are not useful. For example, if a television have an attribute named "Curved" and its value is "No", the title should not include the word "No" alone, because the word "No" without the attribute name means nothing. It is preferable to include "Curved:No".

Your answer should include ONLY the following generated values EXACTLY as they are provided below without any additional formatting or information:
"product attribute keys in original title:"
"product category:"
"product attribute keys:"
"product attribute values:"
"replaced keys:"
"generated title:"

DO NOT use markdown formatting in your response. Do not include any formatting, such as bullet points, numbered lists, bold, italics, headings, subheadings, new lines, tabs, or page breaks.
'''

title_prompt = '''
Taking into account all that has been described so far, generate the content for the following "Context" that is after the 3 dashes "---":

---

Context = {Context}
'''

description_system_instruction = '''Follow these instructions in order:

1. You are a leading digital marketer working for a top retail organization. You are an expert in building detailed and catchy descriptions for the products on your website.
Generate a product description in Spanish that highlights the product's features using the following "Context" information.
If you find a "description" in the given "Context", do NOT reuse it, but make sure you describe any features listed within it in more detail.
If provided with an image, describe the product you see in as much detail as possible and highlight all visible features. You MUST then add this information to the generated description.
If provided with the content of the product's webpage as "Website", extract additional information about the product that is NOT mentioned in "Context". You MUST then add this information to the generated description.
Do NOT include SKU, price, URLs, or any other information that is not directly related to the product's features.
DO NOT use any Markdown syntax, and avoid special characters as much as possible.
The generated description should be at least 500 characters long, preferably at least 1000.

2. I want you to act as a critic with an IQ of 140 and evaluate the description you generated based on the following criteria and points per criterion. Here is the scoring criteria:
Criterion: Repeating sentences depict poor quality and should be scored low.
Criterion: The generated description should strictly be about the provided product. Correct product type, number of items contained in the the product as well as product features such as color should be followed. Any product features that are not present in the input should not be present in the generated description.
Criterion: Hyperbolic text, over promising or guarantees are to be avoided.
Criterion: The generated description must be at least 500 characters in length, but not longer than 5000 characters.
Criterion: The generated description MUST NOT use special characters or any Markdown syntax.

3. Assign a score of 1-5 to the generated description, based on the above criteria:
5 points: The generated description is accurate, well-structured, unique, uses appropriate language, and references the provided "Context", image, and "Website" data only. It is 1000 characters or longer and does not contain any special characters.
4 points: The generated description is accurate and well-structured, but very minor criteria misses are present. It is 700 characters or longer.
3 points: The generated description meets most of the criteria, but may have some issues, such as a few repeating keywords or phrases, or a slightly too formal tone. It is 500 characters or longer and there are special characters present.
2 points: The generated description meets some of the criteria, but has some significant issues, such as inaccurate information, poor structure, or excessive hyperbole.
1 point: The generated description meets very few of the criteria and is of low quality. It is shorter than 500 characters.

Your answer should include ONLY the following generated values EXACTLY as they are provided below without any additional formatting or information:
"description: " the description you generated. Do NOT translate the word "description".
"score: " the score you assigned to the description. Do NOT translate the word "score".
"reasoning: " your reasoning for the score, with examples. Do NOT translate the word "reasoning".

DO NOT use markdown formatting in your response. Do not include any formatting, such as bullet points, numbered lists, bold, italics, headings, subheadings, new lines, tabs, or page breaks.
'''

description_prompt = '''
Taking into account all that has been described so far, generate the content for the following "Context" that is after the 3 dashes "---":

---

Context = {Context}
'''

# %%
# Text-only prompts
product_attributes_system_instruction = '''
You are a leading digital marketer working for a top retail organisation. You are an expert at generating high-performing product listing ad titles and identifying the most important product attributes for influencing a buying decision.

The input will be a text in JSON format that is in a variable called "Context", which provides product information. The content of the "Context" will vary depending on the type of product. For example:

Clothing: The “Context” might include attributes such as size and color.
Detergent: The “Context” might have attributes such as scent.
Television: The “Context” might have attributes such as inches, maximum resolution, and whether it is a Smart TV or not.
Yogurt: The “Context” might include attributes such as flavor, fat content, and whether it is plain or with fruit.
Shampoo: The “Context” might include attributes such as hair type (dry, oily, normal) and whether it is anti-dandruff.

Given the "Context" information for a product, generate all the relevant attributes and product attributes keys of a product. Order attributes, the most relevant, first. The product attributes and product attributes keys should be generated in Spanish.

Prefix the size attribute value with "Tamaño" and replace Small, Medium, Large and X-Large with their abbreviations (S, M, L, XL).
Do not translate to Spanish, things that are internationally accepted in English, like "In-ear", "Notebook", "Cocktail", "email", "streaming", etc.
Do not output special characters as signs, emojis, degree, superscripts, subscripts, etc.
If there was a URL in the Context, do not try to open it. Use the URL to extract additional attributes from the Slug.
DO NOT use markdown formatting in your response. Do not include any formatting, such as bullet points, numbered lists, bold, italics, headings, subheadings, new lines, tabs, or page breaks.

<EXAMPLE>
  INPUT:
    {
      "brand_name": "Samsung",
      "name": "Galaxy Buds 2 Audífonos In-ear Inalámbricos - Blanco",
      "descripcion": "Galaxy Buds 2 Audífonos In-ear Inalámbricos - Blanco",
      "description": "Samsung Galaxy Buds 2. Experimenta un audio con sonido equilibrado, cómodo ajuste, cancelación activa de ruido (ANC) y conectividad perfecta con tu teléfono y reloj. Sonido inmersivo con controles más fáciles y una mejor retroalimentación mientras entrenas, juegas o bailas. Tres micrófonos y una unidad de captura de voz incorporada te ayudan a hacer llamadas más claras, mientras que una solución basada en el aprendizaje automático filtra el sonido no deseado para que puedas compartir mejor tu mundo con familiares y amigos. El diseño de baja protrusión minimiza las interrupciones por el viento para que las llamadas al aire libre sean claras. Dos micrófonos detectan el ruido ambiental mientras que la cancelación activa de ruido bloquea el sonido no deseado. El ruido de fondo puede reducirse hasta un 98 % para mantenerte completamente inmerso en tu música, juegos, audiolibros y tus podcasts favoritos."
    }

  OUTPUT:
    {
      "Marca": "Samsung",
      "Modelo": "Galaxy Buds 2",
      "Color":"Blanco",
      "Tipo": "In-ear",
      "Conectividad": "Inalámbrico",
      "Características":"Sonido balanceado, Controles simples, Ajuste cómodo, Reducción de ruido",
      "Tamaño": "Tamaño S"
    }
</EXAMPLE>

<EXAMPLE>
  INPUT:
    {
      "title": "Monitor Plano Essential 24\" IPS FHD HDMI DPORT S24C330GAL",
      "descripcion": "Marca: SAMSUNG Modelo: S24C330GAL Tipo: Monitor Plano Tamaño: 24\" Resolución: 1,920 x 1,080 (FHD) Tiempo de respuesta: 4ms Tasa de refresco: 100Hz Relación de aspecto: 16:9 Curvatura: No Tecnología Sync: AMD FreeSync Contraste: 1000:1 Brillo: 250 cd/m² Puertos USB: 1 Tipo de panel: IPS Parlantes: No Audio: No Soporte Vesa: SI (75x75) Puertos: 1x HDMI 1x DisplayPort Altura ajustable: No Placa Giratoria: No Ángulo de Visión (RC≥10): 178º(R/L), 178º(U/D) Dimensiones con base: 42.2 Alt x 25.3 Prof x 53.9 anch cm Dimensiones del paquete: 15.6 cm x 39 cm x 67 cm Peso: 4.9Kg Color: Negro P/N: LS24C330GALXZS Condición: Nuevo"
    }

  OUTPUT:
    {
      "Marca": "Samsung",
      "Modelo": "S24C330GAL",
      "Tamaño": "24\"",
      "Tipo": "Monitor Plano",
      "Resolución": "1,920 x 1,080 (FHD)",
      "Panel": "IPS",
      "Conectividad": "HDMI/DPORT",
      "Color": "Negro"
    }
</EXAMPLE>

<EXAMPLE>
  INPUT:
  {
    "brand_name": "Acuenta",
    "name": "Limpiador liquido pisos floral, 5 L",
    "descripcion": "Limpiador liquido pisos floral",
    "description": "Limpiador para Pisos Aroma Floral Marca: acuenta Envase: Botella 5 L País de origen: Chile Producto: Limpiador Estado: Líquido Aroma: Floral Para: Limpiar Pisos Descripción: La formulación del limpiador líquido Acuenta ha sido especialmente desarrollada para limpiar todas las superficies lavables del hogar, desengrasando y aromatizando con una agradable y fresca fragancia. Duración: 24 Meses"
  }

  OUTPUT:
    {
      "Marca": "Acuenta",
      "Producto": "Limpiador líquido pisos",
      "Aroma": "Floral",
      "Tamaño": "5 L",
      "Tipo": "Líquido",
      "Uso": "Pisos"
    }
</EXAMPLE>

<EXAMPLE>
  INPUT:
    {
      "sku": "469",
      "url": "https://www.supermercado.cl/product/sku/123456/gel-para-cabello-frasco-500-g",
      "brand_name": "Acuenta",
      "name": "Gel para Cabello Frasco, 500 g",
      "descripcion": "Gel para Cabello Frasco",
      "description": "Gel para Cabello Marca : acuenta Envase : Frasco 500 g País de origen : Chile Producto : Gel Tipo : Fijador Para : El cabello Duración : 24 Meses Precauciones : Mantener fuera del alcance de los niños Almacenamiento : Mantener en lugar seco y limpio Modo de uso : Colocar un poco de gel sobre sus manos y aplicar sobre el cabello húmedo o seco. Para obtener un aspecto húmedo, no cepillar. Para lograr el aspecto deseado, utilizar una peineta o modelar con las manos. Información adicional : Gel para el cabello el cual entrega brillo, fijación y cuerpo logrando un peinado perfecto. Servicio al consumidor 6006009191"
    }

  OUTPUT:
    {
      "Marca": "Acuenta",
      "Producto": "Gel para Cabello",
      "Envase": "Frasco",
      "Peso": "500 g",
      "País de Origen": "Chile",
      "Tipo": "Fijador",
      "Uso": "Para el cabello",
      "Duración": "24 Meses",
      "Características": "Brillo, Fijación, Cuerpo"
    }

</EXAMPLE>
'''

product_attributes_prompt = '''
Taking into account all that has been described so far, generate the content for the following "Context" that is after the 3 dashes "---":

---

Context = {Context}
'''

title_prompt_textonly_spanish_2 = '''
3. Algunos "product_attribute_values" podrían no entenderse bien si se muestran solos. Por ejemplo, "S" es el tamaño de una prenda de vestir y conviene siempre acompañarlos de su "product_attribute_keys". En un caso así, prefija el valor del atributo de tamaño con "Size" y reemplaza Small, Medium, Large y X-Large por sus abreviaturas (S, M, L, XL), cuando corresponda. Si un valor necesita ir acompañado de su key para ser más entendible, debes incluírlo.

3. Separa los valores de los atributos con comas.

4. Genera un listado de "product_attribute_keys" y sus correspondientes "product_attribute_values"Concatenar todos los "product_attribute_values" en orden y ELIMINA cualquier palabra duplicada que ya hayas concatenado. Si los "product_attribute_values" concatenados superan los 150 caracteres, recorta los valores manteniendo información significativa y evitando cortar palabras. Asegúrate de que no haya palabras duplicadas.

El título generado debe priorizar los atributos de producto más importantes para influir en una decisión de compra. Los atributos más comunes a priorizar incluyen la marca, el tipo de producto, el tamaño y el color. Asegúrate de que el título sea conciso y persuasivo para ser publicado en Merchant Center de Google y en otros catálogos y plataformas de Marketing.

Su respuesta debe incluir SÓLO los siguientes valores generados EXACTAMENTE tal y como se proporcionan a continuación, sin ningún formato o información adicional:
"product attribute keys in original title:"
"product category:"
"product attribute keys:"
"product attribute values:"
"replaced keys:"
"generated title:"

NO utilices el formato markdown en tu respuesta. Utiliza únicamente texto sin formato. No incluyas ningún tipo de formato, como viñetas, listas numeradas, negrita, cursiva, títulos, subtítulos, nuevas líneas, tabulaciones o saltos de página.
Tomando en cuenta todo lo que se ha descrito hasta este momento, genera el contenido para el siguiente "Context" que está después de los 3 guiones "---":

---

Context = {Context}
'''

description_prompt_textonly_spanish = '''Sigue estas instrucciones en el siguiente orden:

1. Eres un experto en marketing digital que trabaja para una organización minorista de primer nivel. Eres un especialista en crear descripciones detalladas y atractivas para los productos en tu sitio web.
Genera una descripción de producto en idioma Castellano que resalte las características del producto utilizando la información proporcionada en el "Context" en formato JSON.
Si encuentras una "description" en el "Context" proporcionado, NO la reutilices. Asegúrate de describir con más detalle cualquier característica listada en el mismo.
NO utilices sintaxis de Markdown y evita, en la medida de lo posible, caracteres especiales.
La descripción generada debe tener al menos 500 caracteres de longitud, preferentemente al menos 1000.

2. Quiero que actúes como un crítico con un IQ de 140 y evalúes la descripción que has generado basándote en los siguientes criterios y puntos por criterio. Aquí están los criterios de evaluación:
Criterio: Las frases repetidas indican baja calidad y deben recibir una puntuación baja.
Criterio: La descripción generada debe estar estrictamente relacionada con el producto proporcionado. El tipo de producto, la cantidad de elementos incluidos en el producto y características como color deben seguirse de manera precisa. Cualquier característica que no esté presente en el input no debe aparecer en la descripción generada.
Criterio: El texto hiperbólico, las promesas exageradas o garantías deben evitarse.
Criterio: La descripción generada debe tener al menos 500 caracteres de longitud, pero no más de 5000.
Criterio: La descripción generada NO DEBE usar caracteres especiales ni sintaxis de Markdown.

3. Asigna una puntuación de 1 a 5 a la descripción generada, según los criterios mencionados:
5 puntos: La descripción generada es precisa, bien estructurada, única, usa un lenguaje apropiado y solo hace referencia a los datos proporcionados en "Context". Tiene 1000 caracteres o más y no contiene caracteres especiales.
4 puntos: La descripción es precisa y bien estructurada, pero presenta pequeños errores menores. Tiene 700 caracteres o más.
3 puntos: La descripción cumple con la mayoría de los criterios, pero puede tener algunos problemas, como algunas palabras o frases repetidas, o un tono ligeramente formal. Tiene 500 caracteres o más y hay caracteres especiales presentes.
2 puntos: La descripción cumple con algunos de los criterios, pero tiene problemas importantes como información inexacta, estructura pobre o excesiva hipérbole.
1 punto: La descripción cumple con muy pocos criterios y es de baja calidad. Tiene menos de 500 caracteres.

Algunas consideraciones adicionales importantes:
- Evita repetir las mismas frases o palabras varias veces. En su lugar, utiliza sinónimos o reformula oraciones para mantener variedad y engagement en la descripción.
- El tono de la descripción debe ser profesional pero atractivo, adecuado para una página de productos de alta gama. Evita usar un lenguaje demasiado informal.
- Si el JSON "Context" está incompleto o falta información clave (por ejemplo, características del producto), haz suposiciones razonables basadas en los datos proporcionados, pero deja claro estas suposiciones en la justificación ("reasoning").

Tu salida debe ser un objeto JSON válido con los siguientes campos:
"description": La descripción que generaste, como string.
"score": La puntuación que asignaste a la descripción, como string.
"reasoning": La justificación de la puntuación, como string con ejemplos específicos.

Asegúrate de que la salida final sea un objeto JSON válido que pueda ser parseado sin errores. Verifica que todas las claves estén presentes y correctamente formateadas, y que no se incluya información o formato adicional.

Devuelve el resultado como un objeto JSON válido con la siguiente estructura. El output JSON debe seguir siempre esta estructura exacta sin desviaciones:
{"description": "<description_text>", "score": "<score_value>", "reasoning": "<reasoning_text>"}

NO utilices formato Markdown, caracteres especiales ni ningún texto plano fuera del objeto JSON. El output debe ser SOLO un objeto JSON válido.

Tomando en cuenta todo lo que se ha descrito hasta este momento, genera el contenido para el siguiente "Context" que está después de los 3 guiones "---":

---

Context = {Context}
'''

# %%
# Description prompt to try to improve descriptions for Disapprovals related to alcohol. Spanish.
description_prompt_textonly_merchant_center_issue_alcoholic_spanish = '''Sigue estas instrucciones en el siguiente orden:

1. Eres un experto en marketing digital que trabaja para una organización minorista de primer nivel. Eres un especialista en crear descripciones detalladas y atractivas para los productos en tu sitio web. Genera una descripción de producto en idioma Castellano que resalte las características del producto utilizando la información proporcionada en el "Context" en formato JSON. Si encuentras una "description" en el "Context" proporcionado, NO la reutilices. Asegúrate de describir con más detalle cualquier característica listada en el mismo. NO utilices sintaxis de Markdown y evita, en la medida de lo posible, caracteres especiales. La descripción generada debe tener al menos 500 caracteres de longitud, preferentemente al menos 1000.

2. Quiero que actúes como un crítico con un IQ de 140 y evalúes la descripción que has generado basándote en los siguientes criterios y puntos por criterio. Aquí están los criterios de evaluación:
Criterio: Las frases repetidas indican baja calidad y deben recibir una puntuación baja.
Criterio: La descripción generada debe estar estrictamente relacionada con el producto proporcionado. El tipo de producto, la cantidad de elementos incluidos en el producto y características como color deben seguirse de manera precisa. Cualquier característica que no esté presente en el input no debe aparecer en la descripción generada.
Criterio: El texto hiperbólico, las promesas exageradas o garantías deben evitarse.
Criterio: La descripción generada debe tener al menos 500 caracteres de longitud, pero no más de 5000.
Criterio: La descripción generada NO DEBE usar caracteres especiales ni sintaxis de Markdown.

3. Asigna una puntuación de 1 a 5 a la descripción generada, según los criterios mencionados:
5 puntos: La descripción generada es precisa, bien estructurada, única, usa un lenguaje apropiado y solo hace referencia a los datos proporcionados en "Context". Tiene 1000 caracteres o más y no contiene caracteres especiales.
4 puntos: La descripción es precisa y bien estructurada, pero presenta pequeños errores menores. Tiene 700 caracteres o más.
3 puntos: La descripción cumple con la mayoría de los criterios, pero puede tener algunos problemas, como algunas palabras o frases repetidas, o un tono ligeramente formal. Tiene 500 caracteres o más y hay caracteres especiales presentes.
2 puntos: La descripción cumple con algunos de los criterios, pero tiene problemas importantes como información inexacta, estructura pobre o excesiva hipérbole.
1 punto: La descripción cumple con muy pocos criterios y es de baja calidad. Tiene menos de 500 caracteres.

4. Además de los criterios anteriores, asegúrate de tomar en cuenta cualquier "merchant_center_issue_name" y "merchant_center_issue_additional_info" presentes en el "Context". Esto te permitirá ajustar la descripción para evitar que el producto sea rechazado nuevamente por Merchant Center. Ten en cuenta las siguientes instrucciones específicas:
Si "merchant_center_issue_name" es "Bebidas alcohólicas" o "alcoholic" toma en cuenta que este es un tema extremadamente sensible dentro de las políticas de Google. Los productos que contienen alcohol tienen estrictas restricciones de promoción debido a normativas locales y a las políticas globales de Google Ads. Por lo tanto, es fundamental que evites cualquier mención que pueda sugerir beneficios para la salud, bienestar o uso del producto en circunstancias inadecuadas, como situaciones laborales, familiares o deportivas. La descripción debe ser completamente neutral y no puede sugerir que el consumo del producto trae ventajas físicas, emocionales o sociales.
Además, es importante que cumplas con todas las normativas locales. Si el "merchant_center_issue_additional_info" menciona restricciones específicas del país, asegúrate de adherirte estrictamente a ellas, ya que Google revisa este tipo de contenido con especial atención. No utilices términos que glorifiquen el consumo de alcohol, como "refrescante", "revitalizante" o "energizante", ya que pueden inducir a una interpretación incorrecta. Si es relevante, resalta el origen del producto, su proceso de elaboración o ingredientes, pero siempre con un enfoque descriptivo y objetivo. Evita cualquier adorno que pueda inducir a equívocos, como afirmaciones sobre cómo el producto mejora la experiencia social o las emociones del consumidor.

Tu salida debe ser un objeto JSON válido con los siguientes campos:
"description": La descripción que generaste, como string.
"score": La puntuación que asignaste a la descripción, como string.
"reasoning": La justificación de la puntuación, como string con ejemplos específicos.

El output JSON debe seguir siempre esta estructura exacta sin desviaciones:
{
  "description": "<description_text>",
  "score": "<score_value>",
  "reasoning": "<reasoning_text>"
}

NO utilices formato Markdown, caracteres especiales ni ningún texto plano fuera del objeto JSON. El output debe ser SOLO un objeto JSON válido.

Tomando en cuenta todo lo que se ha descrito hasta este momento, genera el contenido para el siguiente "Context" que está después de los 3 guiones "---":

---

Context = {Context}
'''

# Description prompt to try to improve descriptions for Disapprovals related to personal hardship. Spanish.
description_prompt_textonly_merchant_center_issue_personal_hardship_spanish = '''Sigue estas instrucciones en el siguiente orden:

1. Eres un experto en marketing digital que trabaja para una organización minorista de primer nivel. Eres un especialista en crear descripciones detalladas y atractivas para los productos en tu sitio web. Genera una descripción de producto en idioma Castellano que resalte las características del producto utilizando la información proporcionada en el "Context" en formato JSON. Si encuentras una "description" en el "Context" proporcionado, NO la reutilices. Asegúrate de describir con más detalle cualquier característica listada en el mismo. NO utilices sintaxis de Markdown y evita, en la medida de lo posible, caracteres especiales. La descripción generada debe tener al menos 500 caracteres de longitud, preferentemente al menos 1000.

2. Quiero que actúes como un crítico con un IQ de 140 y evalúes la descripción que has generado basándote en los siguientes criterios y puntos por criterio. Aquí están los criterios de evaluación:
Criterio: Las frases repetidas indican baja calidad y deben recibir una puntuación baja.
Criterio: La descripción generada debe estar estrictamente relacionada con el producto proporcionado. El tipo de producto, la cantidad de elementos incluidos en el producto y características como color deben seguirse de manera precisa. Cualquier característica que no esté presente en el input no debe aparecer en la descripción generada.
Criterio: El texto hiperbólico, las promesas exageradas o garantías deben evitarse.
Criterio: La descripción generada debe tener al menos 500 caracteres de longitud, pero no más de 5000.
Criterio: La descripción generada NO DEBE usar caracteres especiales ni sintaxis de Markdown.

3. Asigna una puntuación de 1 a 5 a la descripción generada, según los criterios mencionados:
5 puntos: La descripción generada es precisa, bien estructurada, única, usa un lenguaje apropiado y solo hace referencia a los datos proporcionados en "Context". Tiene 1000 caracteres o más y no contiene caracteres especiales.
4 puntos: La descripción es precisa y bien estructurada, pero presenta pequeños errores menores. Tiene 700 caracteres o más.
3 puntos: La descripción cumple con la mayoría de los criterios, pero puede tener algunos problemas, como algunas palabras o frases repetidas, o un tono ligeramente formal. Tiene 500 caracteres o más y hay caracteres especiales presentes.
2 puntos: La descripción cumple con algunos de los criterios, pero tiene problemas importantes como información inexacta, estructura pobre o excesiva hipérbole.
1 punto: La descripción cumple con muy pocos criterios y es de baja calidad. Tiene menos de 500 caracteres.

4. 4. Además de los criterios anteriores, asegúrate de tomar en cuenta cualquier "merchant_center_issue_name" y "merchant_center_issue_additional_info" presentes en el "Context". Esto te permitirá ajustar la descripción para evitar que el producto sea rechazado nuevamente por Merchant Center. Ten en cuenta las siguientes instrucciones específicas:
Si "merchant_center_issue_name" es "Dificultades personales en publicidad personalizada" o "personal_hardship" toma en cuenta que Google aplica restricciones muy estrictas cuando se trata de productos que pueden estar relacionados con problemas de salud, financieros o emocionales de los usuarios. La clave aquí es evitar cualquier tipo de descripción que pueda explotar vulnerabilidades personales. No uses frases que sugieran que el producto podría ayudar a resolver problemas personales como ansiedad, estrés, dificultades financieras o problemas de relaciones. Además, evita apelar a emociones negativas o intentar convencer al lector de que este producto es la solución a problemas personales profundos.

Por ejemplo, si el "merchant_center_issue_additional_info" menciona que el producto ha sido rechazado por dificultades personales, analiza detenidamente si la descripción hace referencia directa o indirecta a posibles beneficios emocionales o psicológicos que el producto no puede ofrecer de forma verificable. Es fundamental que la descripción mantenga un tono neutral, evitando cualquier insinuación de que el producto podría mejorar la situación emocional o financiera del usuario. Usa un lenguaje claro y directo, sin adornos, y evita cualquier promesa que pueda inducir a expectativas irreales o manipuladoras.

Debes asegurarte de que la descripción no sugiera que el producto es una solución para superar alguna forma de crisis personal. Las descripciones deben enfocarse exclusivamente en las características objetivas del producto, sin aludir a posibles resultados emocionales o psicológicos. Es mejor evitar términos como "relajante", "calmante" o "alivia el estrés", a menos que esté clínicamente probado y aceptado por la legislación correspondiente.

Tu salida debe ser un objeto JSON válido con los siguientes campos:
"description": La descripción que generaste, como string.
"score": La puntuación que asignaste a la descripción, como string.
"reasoning": La justificación de la puntuación, como string con ejemplos específicos.

El output JSON debe seguir siempre esta estructura exacta sin desviaciones:
{
  "description": "<description_text>",
  "score": "<score_value>",
  "reasoning": "<reasoning_text>"
}

NO utilices formato Markdown, caracteres especiales ni ningún texto plano fuera del objeto JSON. El output debe ser SOLO un objeto JSON válido.

Tomando en cuenta todo lo que se ha descrito hasta este momento, genera el contenido para el siguiente "Context" que está después de los 3 guiones "---":

---

Context = {Context}
'''

# Description prompt to try to improve descriptions for Disapprovals legal restrictions. Spanish.
description_prompt_textonly_merchant_center_issue_legal_restrictions_spanish = '''Sigue estas instrucciones en el siguiente orden:

1. Eres un experto en marketing digital que trabaja para una organización minorista de primer nivel. Eres un especialista en crear descripciones detalladas y atractivas para los productos en tu sitio web. Genera una descripción de producto en idioma Castellano que resalte las características del producto utilizando la información proporcionada en el "Context" en formato JSON. Si encuentras una "description" en el "Context" proporcionado, NO la reutilices. Asegúrate de describir con más detalle cualquier característica listada en el mismo. NO utilices sintaxis de Markdown y evita, en la medida de lo posible, caracteres especiales. La descripción generada debe tener al menos 500 caracteres de longitud, preferentemente al menos 1000.

2. Quiero que actúes como un crítico con un IQ de 140 y evalúes la descripción que has generado basándote en los siguientes criterios y puntos por criterio. Aquí están los criterios de evaluación:
Criterio: Las frases repetidas indican baja calidad y deben recibir una puntuación baja.
Criterio: La descripción generada debe estar estrictamente relacionada con el producto proporcionado. El tipo de producto, la cantidad de elementos incluidos en el producto y características como color deben seguirse de manera precisa. Cualquier característica que no esté presente en el input no debe aparecer en la descripción generada.
Criterio: El texto hiperbólico, las promesas exageradas o garantías deben evitarse.
Criterio: La descripción generada debe tener al menos 500 caracteres de longitud, pero no más de 5000.
Criterio: La descripción generada NO DEBE usar caracteres especiales ni sintaxis de Markdown.

3. Asigna una puntuación de 1 a 5 a la descripción generada, según los criterios mencionados:
5 puntos: La descripción generada es precisa, bien estructurada, única, usa un lenguaje apropiado y solo hace referencia a los datos proporcionados en "Context". Tiene 1000 caracteres o más y no contiene caracteres especiales.
4 puntos: La descripción es precisa y bien estructurada, pero presenta pequeños errores menores. Tiene 700 caracteres o más.
3 puntos: La descripción cumple con la mayoría de los criterios, pero puede tener algunos problemas, como algunas palabras o frases repetidas, o un tono ligeramente formal. Tiene 500 caracteres o más y hay caracteres especiales presentes.
2 puntos: La descripción cumple con algunos de los criterios, pero tiene problemas importantes como información inexacta, estructura pobre o excesiva hipérbole.
1 punto: La descripción cumple con muy pocos criterios y es de baja calidad. Tiene menos de 500 caracteres.

4. Además de los criterios anteriores, asegúrate de tomar en cuenta cualquier "merchant_center_issue_name" y "merchant_center_issue_additional_info" presentes en el "Context". Esto te permitirá ajustar la descripción para evitar que el producto sea rechazado nuevamente por Merchant Center. Ten en cuenta las siguientes instrucciones específicas:
Si "merchant_center_issue_name" es "Contenido con restricciones legales en publicidad personalizada" o "legal_restrictions" toma en cuenta que este es uno de los puntos más críticos cuando se trata de evitar rechazos en Merchant Center. Los productos que están sujetos a restricciones legales deben cumplir estrictamente con las regulaciones locales y globales impuestas por Google Ads. Si el "Context" contiene el campo "merchant_center_issue_name" relacionado con restricciones legales, debes prestar atención extrema a la precisión de la información proporcionada. No debes mencionar nada que pueda interpretarse como ilegal en el país donde se va a publicar el anuncio. Esto incluye restricciones sobre productos regulados como medicinas, productos farmacéuticos, drogas recreativas, armas, o cualquier otro producto con normativas específicas.

Además, si el "merchant_center_issue_additional_info" proporciona detalles sobre una regulación específica, debes asegurarte de que la descripción no solo cumpla con esa regulación, sino que evite cualquier ambigüedad que podría llevar a la interpretación errónea de que el producto viola alguna ley. Evita absolutamente cualquier declaración o implicación que podría sugerir que el producto puede usarse de manera que infrinja las normativas locales. Si hay dudas sobre las restricciones, es mejor limitarse a características básicas del producto como su tamaño, composición o uso permitido de manera general, sin entrar en detalles que puedan generar problemas legales.

En caso de que el producto esté sujeto a múltiples restricciones legales en distintos territorios, la descripción debe ser genérica y clara, sin entrar en zonas grises que puedan ser malinterpretadas por los reguladores de Google o las autoridades locales. Usa un lenguaje preciso, evitando cualquier tipo de interpretación que pueda sugerir un mal uso o incumplimiento de las normativas legales vigentes. Revisa el "Context" con especial cuidado para detectar cualquier indicio de normativas que afecten la comercialización del producto.

Tu salida debe ser un objeto JSON válido con los siguientes campos:
"description": La descripción que generaste, como string.
"score": La puntuación que asignaste a la descripción, como string.
"reasoning": La justificación de la puntuación, como string con ejemplos específicos.

El output JSON debe seguir siempre esta estructura exacta sin desviaciones:
{
  "description": "<description_text>",
  "score": "<score_value>",
  "reasoning": "<reasoning_text>"
}

NO utilices formato Markdown, caracteres especiales ni ningún texto plano fuera del objeto JSON. El output debe ser SOLO un objeto JSON válido.

Tomando en cuenta todo lo que se ha descrito hasta este momento, genera el contenido para el siguiente "Context" que está después de los 3 guiones "---":

---

Context = {Context}
'''

## Utilities

In [ ]:
def gen_product_attributes(row):
    import vertexai
    from vertexai.generative_models import GenerativeModel, Part, SafetySetting
    from vertexai.preview.prompts import Prompt
    from json import dumps

    print('\n\n########\n#### STARTING SKU: ', row['sku'], row['title'])
    row = row.to_dict()
    context = dumps(row, ensure_ascii=False)
    system_instruction = product_attributes_system_instruction

    variables = [
        {
            "Context": [context],
        },
    ]

    #genai.configure(api_key=API_KEY)
    vertexai.init(project=GCP_PROJECT, location=GCP_LOCATION)
    prompt = Prompt(
        prompt_data=[product_attributes_prompt],
        model_name=MODEL_NAME,
        safety_settings=SAFETY_SETTINGS,
        generation_config=GENERATION_CONFIG_PRODUCT_ATTRIBUTES_TEXT,
        variables=variables,
        system_instruction=[system_instruction]
    )

    contents = prompt.assemble_contents(**prompt.variables[0])
    try:
        # Generate content using the assembled prompt. Change the index if you want
        # to use a different set in the variable value list.
        responses = prompt.generate_content(
            contents=prompt.assemble_contents(**prompt.variables[0]),
            stream=True,
        )
        llm_output = ''
        for response in responses:
            llm_output += response.text
            print(response.text, end="")

    except Exception as e:
        print("ERROR: " + e.__str__())
    generated_product_attributes = llm_output
    return generated_product_attributes

def gen_title(row):
    import vertexai
    from vertexai.generative_models import GenerativeModel, Part, SafetySetting
    from vertexai.preview.prompts import Prompt
    from json import dumps

    row = row.to_dict()
    context = dumps(row, ensure_ascii=False)
    system_instruction = title_system_instruction

    variables = [
        {
            "Context": [context],
        },
    ]

    #genai.configure(api_key=API_KEY)
    vertexai.init(project=GCP_PROJECT, location=GCP_LOCATION)
    prompt = Prompt(
        prompt_data=[title_prompt],
        model_name=MODEL_NAME,
        safety_settings=SAFETY_SETTINGS,
        generation_config=GENERATION_CONFIG_TITLE_TEXT,
        variables=variables,
        system_instruction=[system_instruction]
    )

    contents = prompt.assemble_contents(**prompt.variables[0])
    try:
        # Generate content using the assembled prompt. Change the index if you want
        # to use a different set in the variable value list.
        responses = prompt.generate_content(
            contents=prompt.assemble_contents(**prompt.variables[0]),
            stream=True,
        )
        llm_output = ''
        for response in responses:
            llm_output += response.text
            print(response.text, end="")

    except Exception as e:
        print("ERROR: " + e.__str__())
    generated_title = llm_output
    return generated_title

def gen_description(row):
    import vertexai
    from vertexai.generative_models import GenerativeModel, Part, SafetySetting
    from vertexai.preview.prompts import Prompt
    from json import dumps

    row = row.to_dict()
    context = dumps(row, ensure_ascii=False)
    system_instruction = description_system_instruction

    variables = [
        {
            "Context": [context],
        },
    ]

    #genai.configure(api_key=API_KEY)
    vertexai.init(project=GCP_PROJECT, location=GCP_LOCATION)
    prompt = Prompt(
        prompt_data=[description_prompt],
        model_name=MODEL_NAME,
        safety_settings=SAFETY_SETTINGS,
        generation_config=GENERATION_CONFIG_DESCRIPTION_TEXT,
        variables=variables,
        system_instruction=[system_instruction]
    )

    contents = prompt.assemble_contents(**prompt.variables[0])
    try:
        # Generate content using the assembled prompt. Change the index if you want
        # to use a different set in the variable value list.
        responses = prompt.generate_content(
            contents=prompt.assemble_contents(**prompt.variables[0]),
            stream=True,
        )
        llm_output = ''
        for response in responses:
            llm_output += response.text
            print(response.text, end="")

    except Exception as e:
        print("ERROR: " + e.__str__())
    generated_description = llm_output
    return generated_description

def get_title(generated_title):
    import re
    # Regular expression to extract the generated title after the last occurrence of "generated title"
    pattern = r'generated title:\s*(.*)'

    matches = re.findall(pattern, generated_title)
    if matches:
        generated_title = matches[-1]  # Take the last generated title found
        return(generated_title)
    else:
        return("Generated title not found.")

def get_description(generated_description):
    import re
    # Regular expression to extract the description text after the last occurrence of "description"
    pattern = r'description\s*:\s*(?:"(.*?)"|\'(.*?)\'|(.*?))\s*score'

    match = re.search(pattern, generated_description, re.DOTALL)
    if match:
        generated_description = match.group(1) or match.group(2) or match.group(3)
        return generated_description
    else:
        return "Not found: 'description'"

def create_dataset_and_output_table():
    from google.cloud import bigquery
    # Initialize the BigQuery client
    client = bigquery.Client()

    # Define the project and dataset ID
    dataset_ref = f"{GCP_PROJECT}.{DATASET_NAME}"

    # Create the dataset
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US"
    dataset = client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset {DATASET_NAME} created or already exists.")

    # Define the table ID
    table_ref = f"{dataset_ref}.{OUTPUT_TABLE}"

    # Define the schema
    schema = [
        bigquery.SchemaField("generated_at", "TIMESTAMP", mode="REQUIRED"),
        bigquery.SchemaField("sku", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("id", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("product_attributes", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("generated_title", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("generated_description", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("title", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("description", "STRING", mode="REQUIRED"),
    ]

    # Create the table
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table, exists_ok=True)
    print(f"Table {OUTPUT_TABLE} created or already exists.")

def feedgen2BQ(sku, id, product_attributes, generated_title, generated_description, title, description):
    from google.cloud import bigquery
    from datetime import datetime
    # Initialize the BigQuery client
    client = bigquery.Client()

    # Define the full table reference
    table_ref = f"{GCP_PROJECT}.{DATASET_NAME}.{OUTPUT_TABLE}"

    # Get current timestamp
    generated_at = datetime.utcnow().isoformat()

    # Define the rows to insert
    rows_to_insert = [
        {
            "generated_at": generated_at,
            "sku": sku,
            "id": id,
            "product_attributes": product_attributes,
            "generated_title": generated_title,
            "generated_description": generated_description,
            "title": title,
            "description": description
        }
    ]

    # Insert the rows
    errors = client.insert_rows_json(table_ref, rows_to_insert)

    if errors == []:
        print("Data was successfully inserted.")
    else:
        print(f"Errors occurred: {errors}")

def delete_all_rows():
    from google.cloud import bigquery
    # Initialize the BigQuery client
    client = bigquery.Client()

    # Define the full table reference
    table_ref = f"{GCP_PROJECT}.{DATASET_NAME}.{OUTPUT_TABLE}"

    # Define the SQL query to delete all rows
    query = f"DELETE FROM `{table_ref}` WHERE TRUE"

    # Run the query
    query_job = client.query(query)
    query_job.result()  # Wait for the job to complete

    print(f"All rows from table {OUTPUT_TABLE} have been deleted.")

# Get Input Data

## From BigQuery

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

if BQ_SERVICE_ACCOUNT != '':
    # Create credentials using the service account key file
    credentials = service_account.Credentials.from_service_account_file(BQ_SERVICE_ACCOUNT)
    # Initialize a BigQuery client
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
else:
    # Initialize a BigQuery client without credentials
    client = bigquery.Client(project=GCP_PROJECT)

In [ ]:
# Database query
query = """
-- From Shopping Insider table, search for articles that have less than 5 clicks and have impressions in the last 28 days.
select
  offer_id as sku, title, brand, description, item_url,
  custom_label_0, custom_label_1, custom_label_2, custom_label_3, custom_label_4,
  product_type_l1, product_type_l2,product_type_l3,
  google_product_category_l1, google_product_category_l2, google_product_category_l3,
  --impressions_30_days, clicks_30_days, days_has_impressions
from shopping_insider.product_detailed_materialized
where clicks_30_days < 5 and days_has_impressions > 28
limit 100;
"""


In [ ]:
# Run the query
query_job = client.query(query)
df = query_job.to_dataframe()
df.dropna(axis=1, how='all', inplace=True)
df

## From CSV file (optional)

In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame.
df = pd.read_csv('input_data.csv')
df


# Generate Content

## 1 product

In [ ]:
product = df.iloc[0]

product_attributes = gen_product_attributes(product)

generated_title = gen_title(product)

generated_description = gen_description(product)

title = get_title(generated_title)

description = get_description(generated_description)


In [ ]:
# Debug output
print(product.to_dict().__str__() + '\n' + product_attributes + '\n' + generated_title + '\n' + generated_description + '\n' + title + '\n' + description)


## All products in input dataframe

In [ ]:
import pandas as pd

# Create an empty dataframe with the required columns
generated_feed_df = pd.DataFrame(columns=['sku', 'item_url', 'product_attributes', 'generated_title', 'generated_description', 'title', 'description'])

for i, product in df.sample(5).iterrows():
    try:
        product_attributes = gen_product_attributes(product)
        print(product_attributes)

        generated_title = gen_title(product)
        print(generated_title)

        generated_description = gen_description(product)
        print(generated_description)

        title = get_title(generated_title)
        description = get_description(generated_description)

        # Create a new dataframe with one row and concatenate it to the main dataframe
        new_row = pd.DataFrame({
            'sku': [product['sku']],
            'item_url': [product['item_url']],
            'product_attributes': [product_attributes],
            'generated_title': [generated_title],
            'generated_description': [generated_description],
            'title': [title],
            'description': [description]
        })

        generated_feed_df = pd.concat([generated_feed_df, new_row], ignore_index=True)

    except Exception as e:
        print(f"Error processing product {product['sku']}: {e}")

generated_feed_df


# Output Data

In [ ]:
# Optional. Create dataset and output table in BigQuery
create_dataset_and_output_table()

In [ ]:
# Iterate over each row in the dataframe and use the feedgen2BQ function
for index, row in generated_feed_df.iterrows():
    feedgen2BQ(row['sku'], row['item_url'], row['product_attributes'], row['generated_title'], row['generated_description'], row['title'], row['description'])